In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [3]:

latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir

dfCase = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
dfCase = dfCase.iloc[:-2]
dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['PosPct'] = pd.to_numeric(dfCase['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))
dfCase['Date'] =  pd.to_datetime(dfCase.Date,format='%Y-%m-%d')
testDates = dfCase['Date']

dfAdm = pd.read_csv(latestdir+'/Newly_admitted_over_time.csv',delimiter = ';',dtype=str)
dfAdm['Dato'] = pd.to_datetime(dfAdm['Dato'])
dfAdm['Total'] = pd.to_numeric(dfAdm['Total'])
dfAdm.tail()


dfDea = pd.read_csv(latestdir+'/Deaths_over_time.csv',delimiter = ';',dtype=str)
dfDea = dfDea.iloc[:-1,:]
dfDea['Dato'] = pd.to_datetime(dfDea['Dato'])
dfDea['Antal_døde'] = pd.to_numeric(dfDea['Antal_døde'])
dfDea.tail()

,Dato,Antal_døde
641,2021-12-12,7
642,2021-12-13,9
643,2021-12-14,7
644,2021-12-15,7
645,2021-12-16,0


In [4]:
# Test-figur
fig,ax1 = plt.subplots()
ax1.plot(dfCase.Date,dfCase.NewPositive,'o:',markersize=4,linewidth=1)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# dfCase.NewPositive.iloc[: ]

In [24]:
curDataToUse = dfCase.NewPositive.iloc[:-1]
# curDataToUse = dfCase.NewPositive
dailydiff = curDataToUse.diff()
# curDates = dfCase.Date.values
curDates = dfCase.Date.values[:-1]

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
ax1.plot(curDates,curDataToUse,'b.-',markersize=7,linewidth=1)
ax2.plot([curDates[0],curDates[-1]],[0,0],'k:')
ax2.plot(curDates,dailydiff,'b.-',markersize=7,linewidth=1)

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

curMax = np.max(curDataToUse)
ax1.set_ylim(bottom=0,top=curMax*1.1)
curMaxDiff = np.max(dailydiff)*1.1
ax2.set_ylim([-curMaxDiff,curMaxDiff])

ax1.set_ylabel('Antal tilfælde')
ax2.set_ylabel('Daglig forskel')

firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')
lastDate = curDates[-1]
ax1.set_xlim([firstDate,lastDate])
# ax1.set_xlim([firstDate,lastDate])

# firstDate = np.datetime64('2021-08-01')
# lastDate = np.datetime64('2021-11-15')
# ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'DagligForskel')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
curDataToUse = dfCase.NewPositive.iloc[:-1]
# curDataToUse = dfCase.NewPositive
dailydiff = curDataToUse.diff()
# curDates = dfCase.Date.values
curDates = dfCase.Date.values[:-1]

# fig,(ax1,ax2) = plt.subpblots(2,1,sharex=True)
fig,ax1 = plt.subplots()
ax1.plot(curDates,curDataToUse,'k.-',markersize=5,linewidth=1,label='Data')
ax1.plot(rnTime(curDates,7),rnMean(curDataToUse,7),'b',label='7 dages rullende gennemsnit')
# ax2.plot([curDates[0],curDates[-1]],[0,0],'k:')
# ax2.plot(curDates,dailydiff,'b.-',markersize=7,linewidth=1)

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    # ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=20000)
# ax2.set_ylim([-700,700])

ax1.set_ylabel('Antal tilfælde')
# ax2.set_ylabel('Daglig forskel')

firstDate = np.datetime64('2021-06-01')
# lastDate = np.datetime64('2021-12-01')
ax1.set_xlim([firstDate,lastDate])

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

ax1.legend(loc='upper left')
plt.tight_layout()

plt.savefig(path_figs+'DagligeCases')

ax1.set_yscale('log')
ax1.set_ylabel('Antal tilfælde (logaritmisk akse)')
ax1.grid(axis='y',which='minor')

ax1.set_ylim([100,20000])
plt.tight_layout()

plt.savefig(path_figs+'DagligeCases_Log')

<ipython-input-27-007bf07826b1>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig,ax1 = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
curDataToUse = dfCase.NewPositive.iloc[:-1] 
dailydiff = curDataToUse.diff()
# curDates = dfCase.Date.values
curDates = dfCase.Date.values[:-1]

curErr = np.sqrt(dfCase.Tested.iloc[:-1])

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# ax1.plot(curDates,curDataToUse,'b.-',markersize=7,linewidth=1)
ax1.errorbar(curDates,curDataToUse,yerr=curErr,fmt='b.-',capsize=2,markersize=7,linewidth=0.5)
ax1.plot(rnTime(curDates,7),rnMean(curDataToUse,7),'b')

ax2.plot([curDates[0],curDates[-1]],[0,0],'k:')
ax2.errorbar(curDates,dailydiff,yerr=curErr,fmt='b.-',capsize=2,markersize=7,linewidth=0.5)
ax2.plot(rnTime(curDates,7),rnMean(dailydiff,7),'b')

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

# ax1.set_ylim(bottom=0,top=5000)
# ax2.set_ylim([-900,900])
curMax = np.max(curDataToUse)
ax1.set_ylim(bottom=0,top=curMax*1.1)
curMaxDiff = np.max(dailydiff)*1.1
ax2.set_ylim([-curMaxDiff,curMaxDiff])

ax1.set_ylabel('Antal tilfælde')
ax2.set_ylabel('Daglig forskel')

firstDate = np.datetime64('2021-06-01')
# lastDate = np.datetime64('2021-12-01')
ax1.set_xlim([firstDate,lastDate])
firstDate = np.datetime64('2021-08-01')
# lastDate = np.datetime64('2021-11-15')
ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'DagligForskel_MedUsikkerhed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
curDataToUse = dfCase.NewPositive.iloc[:-1] 
dailydiff = curDataToUse.diff()
# curDates = dfCase.Date.values
curDates = dfCase.Date.values[:-1]

curErr = np.sqrt(dfCase.Tested.iloc[:-1])

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# ax1.plot(curDates,curDataToUse,'b.-',markersize=7,linewidth=1)
ax1.errorbar(curDates,curDataToUse,yerr=curErr,fmt='b.-',capsize=2,markersize=7,linewidth=0.5)
ax1.plot(rnTime(curDates,7),rnMean(curDataToUse,7),'b')

ax2.plot(curDates,np.power(curErr,2),'m.-',markersize=7,linewidth=1)
ax2.plot(rnTime(curDates,7),rnMean(np.power(curErr,2),7),'m')

# ax2.plot([curDates[0],curDates[-1]],[0,0],'k:')
# ax2.errorbar(curDates,dailydiff,yerr=curErr,fmt='b.-',capsize=2,markersize=7,linewidth=0.5)
# ax2.plot(rnTime(curDates,7),rnMean(dailydiff,7),'b')

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=5000)
# ax2.set_ylim([-900,900])

ax1.set_ylabel('Antal tilfælde')
ax2.set_ylabel('Antal tests')

firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')
ax1.set_xlim([firstDate,lastDate])
firstDate = np.datetime64('2021-08-01')
# lastDate = np.datetime64('2021-11-15')
ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'DagligTilfældeOgTest')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
curDataToUse = dfCase.NewPositive.iloc[:-1] 
dailydiff = curDataToUse.diff()
# curDates = dfCase.Date.values
curDates = dfCase.Date.values[:-1]

curErr = np.sqrt(dfCase.Tested.iloc[:-1])

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
ax1.plot(curDates,curDataToUse,'b.-',markersize=5,linewidth=0.5)
# ax1.errorbar(curDates,curDataToUse,yerr=curErr,fmt='b.-',capsize=2,markersize=7,linewidth=0.5)
ax1.plot(rnTime(curDates,7),rnMean(curDataToUse,7),'b')

ax2.plot([curDates[0],curDates[-1]],[0,0],'k:')
# ax2.errorbar(curDates,dailydiff,yerr=curErr,fmt='b.-',capsize=2,markersize=7,linewidth=0.5)
ax2.plot(curDates,dailydiff,'b.-',markersize=5,linewidth=0.5)
ax2.plot(rnTime(curDates,7),rnMean(dailydiff,7),'b')

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=5000)
ax2.set_ylim([-1500,1500])
# curMax = np.max(curDataToUse)
# ax1.set_ylim(bottom=0,top=curMax*1.1)
# curMaxDiff = np.max(dailydiff)*1.1
# ax2.set_ylim([-curMaxDiff,curMaxDiff])

ax2.set_yticks(np.arange(-1500,1510,250))

ax1.set_ylabel('Antal tilfælde')
ax2.set_ylabel('Daglig forskel')

# firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')
firstDate = np.datetime64('2021-08-01')
# lastDate = np.datetime64('2021-11-15')
ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'DagligForskel_MedGennemsnit')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
curDates = dfCase.Date.values
weekDiff = dfCase.NewPositive.values[7:] - dfCase.NewPositive.values[:-7]

dailydiff = dfCase.NewPositive.diff()

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
ax1.plot(dfCase.Date,dfCase.NewPositive,'b.-',markersize=7,linewidth=1)
ax2.plot([curDates[0],curDates[-1]],[0,0],'k:')
ax2.plot(dfCase.Date,dailydiff,'b.-',markersize=7,linewidth=1,label='Daglig forksel')
ax2.plot(curDates[7:],weekDiff,'r.-',markersize=7,linewidth=1,label='Ugentlig forskel')

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=5000)
ax2.set_ylim([-900,900])

ax1.set_ylabel('Antal tilfælde')
ax2.set_ylabel('Forskel')

firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-11-15')
ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'DagligForskel_OgUgentlig')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
dailydiff = dfAdm.Total.diff()
curDates = dfAdm.Dato.values

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
ax1.plot(dfAdm.Dato,dfAdm.Total,'b.-',markersize=7,linewidth=1,label='Data')
ax2.plot([curDates[0],curDates[-1]],[0,0],'k:')
ax2.plot(dfAdm.Dato,dailydiff,'b.-',markersize=7,linewidth=1)

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=80)
ax2.set_ylim([-30,30])

ax1.set_ylabel('Antal nye indlæggelser')
ax2.set_ylabel('Daglig forskel')

firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')
ax1.set_xlim([firstDate,lastDate])

ax1.legend(loc='upper left')

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'DagligForskel_Admissions')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Instead, compare with weekly average

In [13]:
# print(weeklyMeanDates[-10:])
# print(curDates[-10:])
# # Last three days should be ignored


In [14]:
weeklyMean = rnMean(dfCase.NewPositive,7)
weeklyMeanDates = rnTime(dfCase.Date,7)
curDates = dfCase.Date.values
# weeklyMeanDates = rnTime(curDates,7)
weekDiff = dfCase.NewPositive.values[7:] - dfCase.NewPositive.values[:-7]
# weekMeanDiff = dfCase.NewPositive.values[9:] - weeklyMean[:-3]
weekMeanDiff = weeklyMean[7:] - weeklyMean[:-7]

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# ax1.plot(weeklyMeanDates,weeklyMean,'b',linewidth=3)
ax1.plot(dfCase.Date,dfCase.NewPositive,'b.:',markersize=5,linewidth=1,label='Data')
ax1.plot(weeklyMeanDates,weeklyMean,'b',linewidth=3,label='7 dages rullende gennemsnit')

ax2.plot([curDates[0],curDates[-1]],[0,0],'k--')

ax2.plot(curDates[7:],weekDiff,'b.:',markersize=5,linewidth=1,label='Ugentlig forskel')
ax2.plot(rnTime(curDates[7:],7),rnMean(weekDiff,7),'b',linewidth=3,label='7 dages rullende gennemsnit')
# ax2.plot(curDates[9:],weekMeanDiff,'r.-',markersize=7,linewidth=1)
# ax2.plot(weeklyMeanDates[7:],weekMeanDiff,'r.-',markersize=7,linewidth=1)
# ax2.plot(dfCase.Date,dailydiff,'b.-',markersize=7,linewidth=1)

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=3500)
ax2.set_ylim([-900,900])

ax1.set_ylabel('Antal daglige tilfælde')
ax2.set_ylabel('Forskel i antal')

ax1.legend(loc='upper left',fontsize=13)
ax2.legend(loc='lower left',fontsize=13)

firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')
ax1.set_xlim([firstDate,lastDate])

firstDate = np.datetime64('2021-08-01')
lastDate = np.datetime64('2021-11-15')
ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'UgentligForskel')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
curDataToUse = dfCase.NewPositive.iloc[:-1]
# curDates = dfCase.Date.values
curDates = dfCase.Date.values[:-1]
weeklyMean = rnMean(curDataToUse,7)
weeklyMeanDates = rnTime(curDates,7)
# weeklyMeanDates = rnTime(curDates,7)
weekDiff = curDataToUse.values[7:] - curDataToUse.values[:-7]
# weekMeanDiff = curDataToUse.values[9:] - weeklyMean[:-3]
weekMeanDiff = weeklyMean[7:] - weeklyMean[:-7]

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# ax1.plot(weeklyMeanDates,weeklyMean,'b',linewidth=3)
ax1.plot(curDates,curDataToUse,'b.:',markersize=5,linewidth=1,label='Data')
ax1.plot(weeklyMeanDates,weeklyMean,'b',linewidth=3,label='7 dages rullende gennemsnit')

ax2.plot([curDates[0],curDates[-1]],[0,0],'k--')

ax2.plot(curDates[7:],weekDiff,'b.:',markersize=5,linewidth=1,label='Ugentlig forskel')
ax2.plot(rnTime(curDates[7:],7),rnMean(weekDiff,7),'b',linewidth=3,label='7 dages rullende gennemsnit')
# ax2.plot(curDates[9:],weekMeanDiff,'r.-',markersize=7,linewidth=1)
# ax2.plot(weeklyMeanDates[7:],weekMeanDiff,'r.-',markersize=7,linewidth=1)
# ax2.plot(curDates,dailydiff,'b.-',markersize=7,linewidth=1)

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=3500)
ax2.set_ylim([-900,900])

ax1.set_ylabel('Antal daglige tilfælde')
ax2.set_ylabel('Forskel i antal')

ax1.legend(loc='upper left',fontsize=13)
ax2.legend(loc='lower left',fontsize=13)

firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')
ax1.set_xlim([firstDate,lastDate])

# firstDate = np.datetime64('2021-08-01')
# lastDate = np.datetime64('2021-11-15')
# ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'UgentligForskel')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
weeklyMean = rnMean(dfAdm.Total,7)
weeklyMeanDates = rnTime(dfAdm.Dato,7)
curDates = dfAdm.Dato.values
# weeklyMeanDates = rnTime(curDates,7)
weekDiff = dfAdm.Total.values[7:] - dfAdm.Total.values[:-7]
# weekMeanDiff = dfAdm.Total.values[9:] - weeklyMean[:-3]
weekMeanDiff = weeklyMean[7:] - weeklyMean[:-7]

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# ax1.plot(weeklyMeanDates,weeklyMean,'b',linewidth=3)
ax1.plot(dfAdm.Dato,dfAdm.Total,'b.:',markersize=5,linewidth=1,label='Data')
ax1.plot(weeklyMeanDates,weeklyMean,'b',linewidth=3,label='7 dages rullende gennemsnit')

ax2.plot([curDates[0],curDates[-1]],[0,0],'k--')

ax2.plot(curDates[7:],weekDiff,'b.:',markersize=5,linewidth=1,label='Ugentlig forskel')
ax2.plot(rnTime(curDates[7:],7),rnMean(weekDiff,7),'b',linewidth=3,label='7 dages rullende gennemsnit')

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

ax1.set_ylim(bottom=0,top=80)
ax2.set_ylim([-30,30])

ax1.set_ylabel('Antal daglige nyindlæggelser')
ax2.set_ylabel('Forskel i antal')

ax1.legend(loc='upper left',fontsize=13)
ax2.legend(loc='lower left',fontsize=13)

firstDate = np.datetime64('2021-06-01')
lastDate = np.datetime64('2021-12-01')
ax1.set_xlim([firstDate,lastDate])

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

plt.savefig(path_figs+'UgentligForskel_Admissions')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Hospitalsbelægning

In [17]:
# Collect the cases, tests and positive percentage from all the "02_hospitalsbelaegning" files

# Load most recent
latestsubdir = list(os.walk(path_dash))[0][1][-1]

latestsubdir
df_recentHosps = pd.read_csv(path_dash+latestsubdir+'\\Regionalt_DB\\02_hospitalsbelaegning.csv',delimiter = ';',dtype=str,encoding='latin1')
df_recentHosps

# ageGroups = df_recentCasesByAge.transpose().iloc[0]

# dfCase = pd.DataFrame(columns=ageGroups.values)
# dfTest = pd.DataFrame(columns=ageGroups.values)
# dfPosP = pd.DataFrame(columns=ageGroups.values)

allDates = []
allHosps = []

for subdir, dirs, files in os.walk(path_dash):
    if not len(files) == 0:
        latestdir = subdir
        if (latestdir[-12:] == 'Regionalt_DB'): # To ensure it's only once per date
            thisdir = latestdir  
            latestDate = pd.to_datetime(latestdir[-23:-13])
            curdf =  pd.read_csv(latestdir+'\\02_hospitalsbelaegning.csv',delimiter = ';',dtype=str,encoding='latin1')
            
            curSum = curdf.Indlagte.astype(int).sum()
            
            allDates.append(latestDate)
            allHosps.append(curSum)

allDates = np.array(allDates)
allHosps = np.array(allHosps)

In [18]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

lastDateToShow = np.datetime64('2021-11-15')

ax1.plot(allDates,allHosps,'k.:',markersize=7,linewidth=1)

allHospDiff = allHosps[1:] - allHosps[:-1]
ax2.plot([allDates[0],lastDateToShow],[0,0],'k--')
ax2.plot(allDates[1:],allHospDiff,'b.:',markersize=7,linewidth=1,label='Forskel mellem datapunkter')
ax2.plot(rnTime(allDates[1:],7),rnMean(allHospDiff,7),'b',label='7 dages rullende gennemsnit')

ax1.set_ylim(bottom=0)
ax1.set_ylabel('Hospitalsbelægning \n(Antal indlagte)')
ax2.set_ylabel('Forskel')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

ax1.set_ylim(top=400)
ax2.set_ylim(-50,50)
ax1.set_xlim([np.datetime64('2021-07-01'),lastDateToShow])

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')

# ax1.set_ylim(top=1000)
# ax1.set_xlim([np.datetime64('2021-09-01'),np.datetime64('2021-11-15')])

# ax1.grid()
# ax2.grid()
ax2.legend(fontsize=14)
plt.tight_layout()

plt.savefig(path_figs+'Hospitaliseringer')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Sum af tilfælde på bestemte ugedage

In [19]:
numberOfWeeks = 8
lastDate = np.datetime64('2021-11-08')
firstDate = lastDate - np.timedelta64(7*numberOfWeeks,'D')

# dates_period = dfCase[dfCase.Date ]
# cases_period = 

weekDays = ['Mandag','Tirsdag','Onsdag','Torsdag','Fredag','Lørdag','Søndag']

curdf = dfCase[dfCase.Date < lastDate]

weekCount = np.zeros((7,))
for k in range(0,numberOfWeeks):
    
    curDayCount = 0
    for d in range(0,7):
        curDayCount = curdf.NewPositive.values[-k*7 + d]
        weekCount[d] = weekCount[d] + curDayCount
weekCount

fig,ax1 = plt.subplots()

curMean = np.mean(weekCount)
ax1.plot([-1,7],[curMean,curMean],'k:',label='Gennemsnit')
ax1.bar(range(0,7),weekCount)
# ax1.plot([-1,7],[1,1],'k:',label='Gennemsnit')
# ax1.bar(range(0,7),weekCount/curMean)


ax1.set_xticks(range(0,7))
ax1.set_xticklabels(weekDays)

# plt.xticklabel(weekDays)
# plt.plot(weekDays,weekCount)
ax1.set_ylabel('Nye smittetilfælde fundet på ugedag')
ax1.set_xlim([-0.5,6.5])

# plt.ylim([6000,8000])


ax1.legend()

# asdf = firstDate.strftime('%m%d %Y')
ax1.set_title(f'Ugedag for nye smittetilfælde\nPeriode: {pd.to_datetime(firstDate):%d. %B} til {pd.to_datetime(lastDate):%d. %B}')


ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
# ax1.spines['bottom'].set_visible(False)
# ax1.spines['left'].set_visible(False)

plt.tight_layout()

plt.savefig(path_figs+'Ugedag')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …